In [1]:
from urllib.request import Request, urlopen

from bs4 import BeautifulSoup  #parse html page
def getcorpus(url,links):
    request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    response = urlopen(request)
    soup = BeautifulSoup(response, "lxml")
    for a in soup.findAll('a'): #find links
        try:
            url = a['href']
            title = a['title']
            if title == "Older Posts":
                print(title, url)
                links.append(url)
                getcorpus(url, links)
        except:
            title=""
    return title
blog = "https://blog.openai.com/"
links = []
getcorpus(blog, links)

''

In [2]:
links = []
def getcorpus(url):
    request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    response = urlopen(request)
    links = []
    soup = BeautifulSoup(response, "lxml")
    for a in soup.findAll('a'): #find links
        url = a['href']
        if "openai.com" not in url:
            links.append("https://blog.openai.com"+ url)

    return links

def getText(url):
    request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    response = urlopen(request)
    soup = BeautifulSoup(response, "lxml")
    mydivs = soup.findAll("div", {"class":"MainContent"})
    posts = []
    for div in mydivs:
        posts+=map(lambda p:p.text.encode('ascii', errors='replace').decode('ascii').replace("?"," "), div.findAll("p"))
    return (''.join(posts)) #return article as a string
blog = "https://blog.openai.com/"
links = []
links = getcorpus(blog)
links


['https://blog.openai.com/block-sparse-gpu-kernels/',
 'https://blog.openai.comhttps://twitter.com/openai',
 'https://blog.openai.comhttps://www.facebook.com/openai.research',
 'https://blog.openai.com/block-sparse-gpu-kernels/',
 'https://blog.openai.com/learning-a-hierarchy/',
 'https://blog.openai.com/generalizing-from-simulation/',
 'https://blog.openai.com/meta-learning-for-wrestling/',
 'https://blog.openai.com/competitive-self-play/',
 'https://blog.openai.com/nonlinear-computation-in-linear-networks/',
 'https://blog.openai.com/learning-to-model-other-minds/',
 'https://blog.openai.com/baselines-acktr-a2c/',
 'https://blog.openai.com/more-on-dota-2/',
 'https://blog.openai.com/dota-2/',
 'https://blog.openai.com/gathering_human_feedback/',
 'https://blog.openai.com/better-exploration-with-parameter-noise/',
 'https://blog.openai.com/openai-baselines-ppo/',
 'https://blog.openai.com/subscribe',
 'https://blog.openai.comhttps://twitter.com/openai',
 'https://blog.openai.comhttps:

In [3]:
text = []
for link in links:
    try:
        text.append(getText(link))
    except:
        print(link)
        

            

https://blog.openai.comhttps://twitter.com/openai
https://blog.openai.comhttps://www.facebook.com/openai.research
https://blog.openai.comhttps://twitter.com/openai
https://blog.openai.comhttps://www.facebook.com/openai.research
https://blog.openai.comhttps://twitter.com/openai
https://blog.openai.comhttps://www.facebook.com/openai.research


In [4]:
len(link)
text

["We re releasing highly-optimized GPU kernels for an underexplored class of neural network architectures: networks with block-sparse weights. Depending on the chosen sparsity, these kernels can run orders of magnitude faster than cuBLAS or cuSPARSE. We've used them to attain state-of-the-art results in text sentiment analysis and generative modeling of text and images.The development of model architectures and algorithms in the field of deep learning is largely constrained by the availability of efficient GPU implementations of elementary operations. One issue has been the lack of an efficient GPU implementation for sparse linear operations, which we're now releasing, together with initial results using them to implement a number of sparsity patterns. These initial results are promising but not definitive, and we invite the community to join us in pushing the limits of the architectures these kernels unlock.\nDense layers (left) can be replaced with layers that are sparse and wide (ce

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
n_cluster = 3

In [6]:
vectorizer = TfidfVectorizer(max_df=0.5,min_df=2,stop_words='english')

In [7]:
X = vectorizer.fit_transform(text)

In [8]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters =n_cluster ,init='k-means++', max_iter = 100, n_init = 1, verbose =True)


In [9]:
km.fit(X)

Initialization complete
Iteration  0, inertia 72.024
Iteration  1, inertia 37.824
Converged at iteration 1: center shift 0.000000e+00 within tolerance 5.103012e-08


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=3, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=True)

In [10]:
 import numpy as np
np.unique(km.labels_,return_counts=True)

(array([0, 1, 2]), array([10, 19, 16], dtype=int64))

In [11]:
#aggregate text in each cluster
text_d={}
for i, cluster in enumerate(km.labels_):
    oneDocument = text[i]
    if cluster not in text_d.keys():
        text_d[cluster] = oneDocument
    else:
        text_d[cluster] += oneDocument
                

In [12]:
km.labels_

array([2, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2, 1, 1,
       2, 1, 0, 1, 0, 0, 1, 1, 2, 0, 1, 0, 0, 0, 0, 2, 2, 2, 1, 0, 0, 2])

In [13]:
X

<45x1776 sparse matrix of type '<class 'numpy.float64'>'
	with 7676 stored elements in Compressed Sparse Row format>

In [14]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords #collection of stopwords from different languages
from nltk.probability import FreqDist
from collections import defaultdict
from heapq import nlargest
from string import punctuation #punctuation since they are considered as tokens

In [15]:
_stopwords = set(stopwords.words('english') + list(punctuation)) #get stopwords and punctuation

In [16]:
from nltk.stem.lancaster import LancasterStemmer
st = LancasterStemmer()

In [37]:
keywords = {}
counts = {}

for cluster in range(n_cluster):
    word_sent = word_tokenize(text_d[cluster].lower())   
    word_sent=[word for word in word_sent if word not in _stopwords]
    
    %stemmedWords=[st.stem(word) for word in word_sent]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(100,freq,key=freq.get)
    counts[cluster]=freq

In [38]:
unique_keys={}
for cluster in range(n_cluster):
    print(cluster)
    other_clusters=list(set(range(n_cluster))-set([cluster]))
    keys_other_clusters=set(keywords[other_clusters[0]]).union(set(keywords[other_clusters[1]]))

    unique=set(keywords[cluster])-keys_other_clusters

    unique_keys[cluster]= nlargest(10, unique, key=counts[cluster].get)
   

0
1
2


In [39]:
unique_keys

{0: ['machine',
  'people',
  'problems',
  'defense',
  'attacker',
  'paper',
  'output',
  'everyone',
  'event',
  'masking'],
 1: ['agents',
  'environments',
  'environment',
  'reward',
  'rl',
  'games',
  'actions',
  'performance',
  'noise',
  'algorithm'],
 2: ['images',
  'data',
  'x',
  'generative',
  'sparse',
  'world',
  'networks',
  'robot',
  'min',
  'kernels']}

In [40]:
unique_keys[cluster]= nlargest(10, unique, key=counts[cluster].get)

In [41]:
unique_keys

{0: ['machine',
  'people',
  'problems',
  'defense',
  'attacker',
  'paper',
  'output',
  'everyone',
  'event',
  'masking'],
 1: ['agents',
  'environments',
  'environment',
  'reward',
  'rl',
  'games',
  'actions',
  'performance',
  'noise',
  'algorithm'],
 2: ['images',
  'data',
  'x',
  'generative',
  'sparse',
  'world',
  'networks',
  'robot',
  'min',
  'kernels']}

In [45]:
from urllib.request import Request, urlopen
from sklearn.neighbors import KNeighborsClassifier
from bs4 import BeautifulSoup  #parse html page
url = "https://futureofwork.azuronaut.com/workplace-by-facebook-and-sharepoint-better-together-b703203211e7"
#get the webpage
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})  #tell that the scraper is a web browser
webpage = urlopen(req).read().decode('utf8', 'ignore')
soup = BeautifulSoup(webpage, "lxml")

#special characters are due to encoding
text = text.encode('ascii', errors='replace').decode('ascii')
text.replace("?", " ")


'Regan CollinsBlockedUnblockFollowFollowingCEO at AzuronautNov 27Workplace by Facebook and SharePoint   Better Together.Image credit: Workplace by FacebookTwo of the world s biggest brands in technology and business are proving that working together provides the best solution for their Enterprise clients.The theory writers have not had time to put pen to paper on this unwritten and somewhat undisclosed relationship, but as the CEO of Azuronaut   a Workplace by Facebook and a Microsoft Gold partner   I ve been sitting in the catbird seat.From my vantage point it s clear that both Microsoft and Workplace by Facebook can give big benefit to businesses by allowing organisations to use the best of both platforms.Whole new worldOrganisations previously tried to meet their needs by using  one size fits  all software solutions. Now   with SaaS implementation times and pricing methodologies   businesses can use multiple on-point cloud technologies, integrate them and reap the benefits. So when 

In [50]:
## k-neighbours classifier
classifier = KNeighborsClassifier(n_neighbors=10)
classifier.fit(X,km.labels_) #training

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [51]:
#represent article with tf_IDF form
test = vectorizer.transform([text])

In [52]:
test

<1x1776 sparse matrix of type '<class 'numpy.float64'>'
	with 156 stored elements in Compressed Sparse Row format>

In [53]:
classifier.predict(test)

array([2])